In [ ]:
# 데이터 전처리 코드입니다.

"""
- EDA 인사이트
1) EDA를 통해 클래스 별 불균형 확인 (가장 큰건 300장이 넘어가지만, 8장의 이미지를 가지는 클래스도 존재)
-> 불균형을 고려한 데이터 증강 필요성

추가하고 싶은 것) Train 데이터와 test 데이터의 분포 비교. (이거 해도 되는지 모르겠음..)
-> intensity, 주파수 영역 분석해서 분포가 비슷한걸 파악함 -> 이미지 분포를 크게 바꾸지 않는 증강만 고려


- 전처리 과정: 데이터 불균형을 고려한 데이터 증강
1) 클래스가 새, 자동차, 비행기이므로 좌우 반전 증강 가능.

2) 클래스가 새, 자동차, 비행기이므로 현실 세계를 고려하여 45도 이내의 시계/ 반시계 방향 랜덤 회전 
-> 이렇게 진행 시 모든 데이터 3배 증가함.

3) 이렇게 증강해도 300장이 되지 않는 데이터들에 대하여 45도 이내의 시계/ 반시계 방향 랜덤 회전 또 진행
-> 가장 작은 클래스도 300장이 넘어가도록 하여 데이터 불균형 해소

"""

In [1]:
import os
import random
from PIL import Image, ImageOps
from torchvision.transforms import functional as F

In [2]:
# 데이터 경로 설정
train_dir = "/test/final_exam/challenge/train"
augmented_dir = "dogun_convnext_tiny_try111/dl-bootcamp-2025-challenge_aug/train"
os.makedirs(augmented_dir, exist_ok=True)


In [4]:
# 증강 함수 정의
def augment_images(class_path, save_path, min_count=300):
    images = [f for f in os.listdir(class_path) if f.endswith(".jpg") or f.endswith(".png")]
    
    # 원본 이미지 불러오기
    image_paths = [os.path.join(class_path, img) for img in images]
    augmented_images = []

    # 좌우 반전
    for img_path in image_paths:
        with Image.open(img_path) as img:
            flipped = ImageOps.mirror(img)
            augmented_images.append(flipped)
            augmented_images.append(img.copy())

    # 랜덤 회전 (시계방향 & 반시계방향 45도 이내)
    final_images = []
    for img in augmented_images:
        for _ in range(2):
            angle = random.uniform(-45, 45)
            rotated = img.rotate(angle)
            final_images.append(rotated)

    # 이미지 저장 (최소 100장 확보)
    while len(final_images) < min_count:
        for img in augmented_images:
            angle = random.uniform(-45, 45)
            rotated = img.rotate(angle)
            final_images.append(rotated)
            if len(final_images) >= min_count:
                break

    # 저장
    os.makedirs(save_path, exist_ok=True)
    for idx, img in enumerate(final_images):
        img.save(os.path.join(save_path, f"aug_{idx}.jpg"))


In [5]:
# 클래스별 증강 실행
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        save_path = os.path.join(augmented_dir, class_name)
        augment_images(class_path, save_path)

print("Data augmentation complete.")

Data augmentation complete.
